### Simple EDA

In [1]:
import numpy as np
import pandas as pd

In [2]:
### models ###
# Support Vector machine
from sklearn import svm
# Naive Bayes
import sklearn.naive_bayes as nb
# Logistic Regression
from sklearn.linear_model import LogisticRegression
# decision tree
from sklearn.tree import DecisionTreeClassifier
# random forest 
from sklearn.ensemble import RandomForestClassifier

In [3]:
train = pd.read_csv("train.csv")
train.head()

,Insult,Date,Comment
0,1,20120618192155Z,"""You fuck your dad."""
1,0,20120528192215Z,"""i really don't understand your point.\xa0 It ..."
2,0,NaN,"""A\\xc2\\xa0majority of Canadians can and has ..."
3,0,NaN,"""listen if you dont wanna get married to a man..."
4,0,20120619094753Z,"""C\xe1c b\u1ea1n xu\u1ed1ng \u0111\u01b0\u1edd..."


In [4]:
train.shape

(3947, 3)

In [5]:
train[['Comment','Insult']].groupby('Insult').count()

,Comment
Insult,
0,2898
1,1049


In [6]:
# 0. 라벨과 댓글 쪼개기
label, comments = train['Insult'],train['Comment']

### Preprocessing

In [7]:
from nltk import sent_tokenize
import re
from re import sub
import string
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer


import nltk
from nltk.corpus import stopwords
'''
1. Text Cleansing(lower case, removal)
2. Tokenization
3. Stemming(줄기어로 바꾸기)
4. Lemmatization(형태소 부석)
5. Stop words removal()
6. Rare words removal()
'''

'\n1. Text Cleansing(lower case, removal)\n2. Tokenization\n3. Stemming(줄기어로 바꾸기)\n4. Lemmatization(형태소 부석)\n5. Stop words removal()\n6. Rare words removal()\n'

In [8]:
def stemming(x):
    #st = LancasterStemmer()
    st = WordNetLemmatizer()
    words=x.strip().split()
    st3=SnowballStemmer("english")
    return [st3.stem(st.lemmatize(x)) for x in words]

def remove_special_char(x):
    # this will replace all punctuations with spaces
    punc = string.punctuation.replace("-", "")
    punc = punc.replace("'", "")
    pat= r"[{}]".format(punc)
    x=re.sub(pat, " ", x)
    
    # this will replace all digits with None
    x=re.sub(pattern=r"\d", repl=r" ", string=x)
    
    # this will strip extra white spaces
    return " ".join( i for i in stemming(x))


In [9]:
# 1
comments=comments.str.lower()

In [10]:

# 2, 3, 4
comments_tran = comments.apply(remove_special_char)
comments.iloc[1], comments_tran.iloc[1]

('"i really don\'t understand your point.\\xa0 it seems that you are mixing apples and oranges."',
 "i realli don't understand your point \\xa it seem that you are mix appl and orang")

In [11]:

# 5, 6
stopWords = set(stopwords.words('english'))
type(stopWords), len(stopWords)


(set, 179)

In [13]:
tf = text.TfidfVectorizer(stop_words=list(stopWords), ngram_range=(1, 1))
X = tf.fit_transform(comments_tran)
print(X.shape)

(3947, 11775)


In [14]:
vocab=list((tf.vocabulary_).keys())

### Modeling

In [15]:
import sklearn

# to split data
from sklearn.model_selection import train_test_split, ShuffleSplit
import sklearn.feature_extraction.text as text

### models ###
# Support Vector machine
from sklearn import svm
# Naive Bayes
import sklearn.naive_bayes as nb
# Logistic Regression
from sklearn.linear_model import LogisticRegression
# decision tree
from sklearn.tree import DecisionTreeClassifier
# random forest 
from sklearn.ensemble import RandomForestClassifier

In [16]:
(X_train, X_val,label_train, label_val) = train_test_split(X, label, test_size=.2)
X_train.shape, X_val.shape

((3157, 11775), (790, 11775))

#### linear classfication model

In [17]:
def model_LR():
    # creating classifier
    clf = LogisticRegression(tol=1e-8, penalty='l2', C=2)
    # training classifier
    clf.fit(X_train, label_train)
    # model type
    print("Model: ",type(clf))
    # Predicting probabilities
    p = clf.predict_proba(X_val)
    return (clf.predict(X_val),p)

def model_SVM():
    # creating classifier
    clf = svm.LinearSVC(penalty='l2', loss='squared_hinge',tol=1e-8)
    # training classifier
    clf.fit(X_train, label_train)
    # model type
    print("Model: ",type(clf))
    return clf.predict(X_val)

# Bernoulli Naive Baiyes
def model_BernoulliNB():
    # creating classifier
    clf = nb.BernoulliNB(alpha=1.0, binarize=0.0)
    # training classifier
    clf.fit(X_train, label_train)
    # model type
    print("Model: ",type(clf))
    # Predicting probabilities
    p = clf.predict_proba(X_val)
    return (clf.predict(X_val),p)

#### non-linear classification model

In [18]:
# Random Forest Classifier
def model_RF():
    # creating classifier
    clf = RandomForestClassifier(n_estimators=100)
    # training classifier
    clf.fit(X_train, label_train)
    # model type
    print("Model: ",type(clf))
    # Predicting probabilities
    p = clf.predict_proba(X_val)
    return (clf.predict(X_val),p)

In [19]:
def model_DT():
    # creating classifier
    clf = DecisionTreeClassifier(max_depth=100)
    # training classifier
    clf.fit(X_train, label_train)
    # model type
    print("Model: ",type(clf))
    # Predicting probabilities
    p = clf.predict_proba(X_val)
    return (clf.predict(X_val),p)

### Evaluation

In [20]:
# ROC and AUC score
from sklearn.metrics import roc_auc_score as auc_score
# Confusion Matrix
from sklearn.metrics import confusion_matrix

In [21]:
def model_evaluation(model,label_test):
    #accuracy=np.mean(model == label_test)
    #print("%.4f"%np.mean(model == label_test))
    # confusion matrix:
    cm = confusion_matrix(label_test, model, labels=None, sample_weight=None)
    tp, fn, fp, tn = cm[0][0], cm[0][1], cm[1][0], cm[1][1]
    precision= float(tp)/(tp+fp)
    recall =  float(tp)/(tp+tn)
    accuracy = np.mean(model == label_test)
    print_results (precision, recall, accuracy)
    return accuracy
    

def print_results (precision, recall, accuracy):
    banner = "Here is the classification report"
    print ('\n',banner)
    print ('=' * len(banner))
    print ('{0:10s} {1:.1f}'.format('Precision',precision*100))
    print ('{0:10s} {1:.1f}'.format('Recall',recall*100))
    print ('{0:10s} {1:.1f}'.format('Accuracy',accuracy*100))
    
    
    #print("*****PRECISION****")
    #print("%.4f"%(tp/(tp+fp)))
    #print("*****RECALL****")
    #print("%.4f"%(tp/(tp+tn)))
    #return accuracy

In [22]:
# model prediction
clf_LR, p = model_LR()
# model evaluation
acc_LR = model_evaluation(clf_LR, label_val)

print('{0:10s} {1:.1f}'.format('AUC Score',auc_score(label_val, p[:,1])))

Model:  <class 'sklearn.linear_model._logistic.LogisticRegression'>

 Here is the classification report
Precision  82.6
Recall     87.0
Accuracy   82.0
AUC Score  0.9


In [24]:
# model prediction
clf_SVM = model_SVM()
# model evaluation
acc_SVM = model_evaluation(clf_SVM, label_val)

Model:  <class 'sklearn.svm._classes.LinearSVC'>

 Here is the classification report
Precision  86.5
Recall     82.5
Accuracy   84.4


/Users/hee/git/metaverse_patrol/metaverse/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


In [25]:

# model prediction
clf_NB,p=model_BernoulliNB()
# model evaluation
acc_NB = model_evaluation(clf_NB, label_val)

print ('{0:10s} {1:.1f}'.format('AUC Score',auc_score(label_val, p[:,1])*100))

Model:  <class 'sklearn.naive_bayes.BernoulliNB'>

 Here is the classification report
Precision  78.4
Recall     92.3
Accuracy   77.5
AUC Score  81.1


In [26]:
# model prediction
clf_RF,p=model_RF()
# model evaluation
acc_RF = model_evaluation(clf_RF, label_val)

print ('{0:10s} {1:.1f}'.format('AUC Score',auc_score(label_val, p[:,1])*100))

Model:  <class 'sklearn.ensemble._forest.RandomForestClassifier'>

 Here is the classification report
Precision  81.4
Recall     87.7
Accuracy   79.5
AUC Score  83.5


In [27]:
# model prediction
clf_DT,p=model_DT()
# model evaluation
acc_DT = model_evaluation(clf_DT, label_val)

print ('{0:10s} {1:.1f}'.format('AUC Score',auc_score(label_val, p[:,1])*100))

Model:  <class 'sklearn.tree._classes.DecisionTreeClassifier'>

 Here is the classification report
Precision  84.2
Recall     81.8
Accuracy   76.6
AUC Score  66.4


In [28]:
# Accuracy for all Models
accuracy_normal=[acc_LR, acc_SVM, acc_NB, acc_RF, acc_DT]
accuracy_normal=[('{0:2f}'.format(i*100)) for i in accuracy_normal]

In [53]:
models = ['Logistic Regression', 'Support Vector Machine', 'Naive Bayes', 'Random Forest', 'Decision Tree']
for name, acc in zip(models, accuracy_normal):
    print('{0:<25s} {1:>5s}'.format(name, acc))

Logistic Regression       82.025316
Support Vector Machine    84.430380
Naive Bayes               77.468354
Random Forest             79.493671
Decision Tree             76.582278


### Cross Validation